### Load environment variables

In this step, we use the dotenv package to load environment variables from a .env file. This approach helps manage sensitive configuration details like API keys and service credentials without hardcoding them in the code.

Key Points:

dotenv: Automatically loads environment variables defined in a .env file into process.env.

Access Environment Variables: The process.env object is used to access these variables in the application.

In [ ]:
import dotenv from 'dotenv';
dotenv.config();
 
console.log(process.env.AICORE_SERVICE_KEY); 

### Create a New Orchestration Configuration
In this step, we define a function to create an orchestration configuration using the ConfigurationApi from the SAP AI SDK. This configuration integrates various parameters needed for orchestration, such as the executable ID and scenario ID.

Key Points:

ConfigurationApi: Provides methods for interacting with the SAP AI SDK's configuration services.

parameterBindings: Specifies the parameters used for orchestration.

In [3]:
import { ConfigurationApi } from '@sap-ai-sdk/ai-api';

const RESOURCE_GROUP = 'grounding'; // Please change to your desired resource group

// Create orchestration configuration using ConfigurationApi
async function createOrchestrationConfiguration() {
  try {
    const response = await ConfigurationApi
      .configurationCreate({
        name: 'orchestration-config', // Choose a meaningful name
        executableId: 'orchestration', // Orchestration executable ID
        scenarioId: 'orchestration', // Orchestration scenario ID
      }, {'AI-Resource-Group': RESOURCE_GROUP}).execute();

      return response;
  } catch (error: any) {
      // Handle API errors
      console.error('Configuration creation failed:', error.stack);
  }
}

const configuration = await createOrchestrationConfiguration();
console.log(configuration?.message); // Print the configuration response message

Configuration created


### Deployment of orchestration
This step involves creating a deployment using the specified configuration and resource group. The deployment is handled via the DeploymentApi, which streamlines the process of activating the orchestration setup.

Key Points:

DeploymentApi: Used for initiating the deployment based on the given configuration.

createDeployment Function: This function handles the API call to create the deployment.

In [4]:
import { DeploymentApi } from '@sap-ai-sdk/ai-api'; 

// Create Orchestration deployment using DeploymentApi
async function createOrchestrationDeployment() { 
 // Extract the configuration ID from the result of the previous step 
  const configurationId = configuration.id;

 try { 
 const response = await DeploymentApi
      .deploymentCreate(
        { configurationId }, 
        { 'AI-Resource-Group': RESOURCE_GROUP }
      ).execute();   

    return response;
 } catch (error: any) { 
    console.error('Deployment creation failed:', error.stack);
 } 
} 
 
const deployment = await createOrchestrationDeployment();
console.log(deployment?.message) // Print the deployment creation response

Deployment scheduled.


### Basic Orchestration Pipeline

In [5]:
const cvContent = await Deno.readTextFile('./cv.txt');

### Templating

Explanation of Templating Code

This code defines a template for an AI assistant using orchestration configuration. The `Template` object is set up with system and user messages to guide the assistant’s response behavior. 

Key Components:
- **SystemMessage**: Sets a predefined instruction for the AI assistant. This message typically includes the assistant's role and any specific guidelines it should follow.
- **UserMessage**: Represents the user's input and how it is structured in the conversation.
  
In this revised prompt, only queries are passed to the assistant without any additional context. The AI is expected to respond based solely on the provided input.


###  Define the LLM 

The LLM class is used to configure and initialize a model for generating text based on specific parameters. In this example, we'll use the model to perform the content creation task.

ℹ️Note that virtual deployment of the model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

### Generate Responses 

This step outlines the process of generating responses for a set of queries using a defined model. The generateResponses function executes queries to gather AI-generated responses.

Key Points:

Query Execution: Uses OrchestrationClient to generate responses for each query.

In [6]:
import { OrchestrationClient } from '@sap-ai-sdk/orchestration';

const orchestrationClient = new OrchestrationClient({
  promptTemplating: {
    model: {
      name: 'gpt-4o'
    },
    prompt: {
      template: [
        {
          role: 'system',
          content:
            'You are a helpful AI assistant for HR. Summarize the following CV in 10 sentences, focusing on key qualifications, work experience, and achievements. Include personal contact information, organizational history, and personal interests.'
        },
        {
          role: 'user',
          content: 'Candidate Resume:\n{{?candidate_resume}}'
        }
      ]
    }
  }
});

const response = await orchestrationClient.chatCompletion({
  placeholderValues: {
    candidate_resume: cvContent
  }
});
console.log(response.getContent());

John Doe is a skilled data scientist with over three years of experience in data analysis, statistical modeling, and machine learning, aiming to utilize his proficiency in predictive modeling and data visualization to inform decision-making at prospective employers. He holds a Master's degree in Data Science from the University of California, Berkeley, and a Bachelor's degree in Computer Science from UCLA. John is proficient in several programming languages including Python, R, SQL, and Java, and is experienced with data analysis and visualization tools such as Pandas, NumPy, Matplotlib, Seaborn, and Tableau, alongside machine learning libraries like Scikit-learn, TensorFlow, and Keras. His professional experience includes working as a Data Scientist at DataCorp Inc., where he developed predictive models that increased ROI by 20% and visualized KPIs using Tableau to improve stakeholder decision-making. Previously, as a Data Analyst Intern at Analytics Solutions, he contributed to busin

In [ ]:
response